<a href="https://colab.research.google.com/github/santoshrsarangi/tensorflow/blob/main/NLP/RNN_IMDB_Movie_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers


In [2]:
from keras.layers import TextVectorization

In [3]:
text_vectorization = TextVectorization(output_mode="int")

In [4]:
import re
import string

In [5]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [6]:
from numpy import string_
def custom_standardization_fn(string_tensor):
  lowercase_string = tf.strings.lower(string_tensor)
    

In [25]:
dataset = [
    "hello, this, is a sample text which I'd like to test on Text Vecrotization leayer of keras. This should/, help in NLP"
]

In [26]:
text_vectorization.adapt(dataset)

In [27]:
vocab=text_vectorization.get_vocabulary()

In [28]:
encoded_sentense = text_vectorization("Hello, how far have you learned NLP Text Vectorization using Keras")

In [29]:
print(encoded_sentense)

tf.Tensor([20  1  1  1  1  1 12  3  1  1 15], shape=(11,), dtype=int64)


In [30]:
inverse_vocab = dict(enumerate(vocab))

In [31]:
decoded_sentense = " ".join(inverse_vocab[int(i)] for i in encoded_sentense)

In [32]:
print(decoded_sentense)

hello [UNK] [UNK] [UNK] [UNK] [UNK] nlp text [UNK] [UNK] keras
